Vous trouverez dans notre archive seulement nos fichiers de configuration car notre archive globale était trop volumineuse.

In [1]:
!pip install OpenNMT-py

     |████████████████████████████████| 216 kB 5.3 MB/s 
     |████████████████████████████████| 56 kB 1.7 MB/s 
     |████████████████████████████████| 73 kB 1.1 MB/s 
     |████████████████████████████████| 16.3 MB 79 kB/s 
     |████████████████████████████████| 1.2 MB 36.1 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


## Préparation des données

**Q1** : Que contiennent les fichiers _IWSLT10_BTEC.train.en.txt_ et _IWSLT10_BTEC.train.fr.txt_ ?

Dans le fichier _IWSLT10_BTEC.train.en.txt_ on trouve des phrases en anglais. Sur le fichier _IWSLT10_BTEC.train.fr.txt_ on trouve les mêmes phrases correspondantes en français.

**Q2** : Pour chacun des fichiers dans les répertoires train, dev et test, exécutez la commande suivante : `cat file_in.txt | awk -F '\' '{print $NF}' > file_out.clean.txt`. Pour chaque fichier clean créé, exécutez la commande suivante (en changeant LANG par en ou fr selon la langue dans laquelle est le fichier) : `tokenizer.perl -l LANG -lc < ./file_in.LANG.clean.txt > file_out.LANG.tok.txt`. Quelles différences voyez-vous entre les fichiers _.clean.txt_ et _.tok.txt_ ? Quel est l’intérêt de l’opération effectuée ?

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/tp_nmt/tp_nmt/

/content/drive/MyDrive/tp_nmt/tp_nmt


In [ ]:
!cat BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.txt | awk -F '\' '{print $NF}' > BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.clean.txt
!cat BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.txt | awk -F '\' '{print $NF}' > BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.clean.txt
!cat BTEC-en-fr/train/IWSLT10_BTEC.train.en.txt | awk -F '\' '{print $NF}' > BTEC-en-fr/train/IWSLT10_BTEC.train.en.clean.txt
!cat BTEC-en-fr/train/IWSLT10_BTEC.train.fr.txt | awk -F '\' '{print $NF}' > BTEC-en-fr/train/IWSLT10_BTEC.train.fr.clean.txt
!cat BTEC-en-fr/test/IWSLT09_BTEC.testset.en.txt | awk -F '\' '{print $NF}' > BTEC-en-fr/test/IWSLT09_BTEC.testset.en.clean.txt
!cat BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.txt | awk -F '\' '{print $NF}' > BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.clean.txt

In [ ]:
!perl tokenizer.perl -l en -lc < BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.clean.txt > BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt
!perl tokenizer.perl -l fr -lc < BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.clean.txt > BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt
!perl tokenizer.perl -l en -lc < BTEC-en-fr/train/IWSLT10_BTEC.train.en.clean.txt > BTEC-en-fr/train/IWSLT10_BTEC.train.en.tok.txt
!perl tokenizer.perl -l fr -lc < BTEC-en-fr/train/IWSLT10_BTEC.train.fr.clean.txt > BTEC-en-fr/train/IWSLT10_BTEC.train.fr.tok.txt
!perl tokenizer.perl -l en -lc < BTEC-en-fr/test/IWSLT09_BTEC.testset.en.clean.txt > BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt
!perl tokenizer.perl -l fr -lc < BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.clean.txt > BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt

Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: fr
Number of threads: 1
Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: fr
Number of threads: 1
Tokenizer Version 1.1
Language: en
Number of threads: 1
Tokenizer Version 1.1
Language: fr
Number of threads: 1


Réponse à la question 2 :

Les fichiers .tok ont géré la ponctuation : des espaces ont été rajoutés avant chaque ponctuation (les virgules, les points, ...)
Les apostrophes ont été remplacées par leur code HTML \&apos;.

Cela permettra de mieux traiter l'identification des mots.

**Q3** : Pourquoi avoir un corpus séparé en 3 parties (train, dev et test) ? A quoi vont servir chacun de ces fichiers ? (Note: bien expliquer la différence d'utilisation entre dev et test et la raison de cela)

Le train set contient les données sur lesquelles le réseau apprend.

Le dev set contient les données permettant d'ajuster les paramètres du réseau durant l'apprentissage.

Le test set contient des données que le réseau n'a jamais vu et qui permettent d'évaluer la précision de notre réseau.

## Création d’un modèle de TA

In [ ]:
!onmt_build_vocab -config config-base.yaml -n_sample -1

Corpus train's weight should be given. We default it to 1 for you.
[2022-02-15 11:34:16,441 INFO] Counter vocab from -1 samples.
[2022-02-15 11:34:16,442 INFO] n_sample=-1: Build vocab on full datasets.
[2022-02-15 11:34:16,459 INFO] train's transforms: TransformPipe()
[2022-02-15 11:34:16,865 INFO] Counters src:9978
[2022-02-15 11:34:16,865 INFO] Counters tgt:8194
Traceback (most recent call last):
  File "/usr/local/bin/onmt_build_vocab", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/build_vocab.py", line 71, in main
    build_vocab_main(opts)
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/build_vocab.py", line 55, in build_vocab_main
    save_counter(src_counter, opts.src_vocab)
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/build_vocab.py", line 44, in save_counter
    check_path(save_path, exist_ok=opts.overwrite, log=logger.warning)
  File "/usr/local/lib/python3.7/dist-packages/onmt/utils/misc.py", line 17, in 

**Q4** : pourquoi un système de TA a-t-il besoin d'un vocabulaire ? pourquoi est-ce que l'on utilise un vocabulaire basé sur les données d'apprentissage et non-pas un vocabulaire quelconque, plus universel ?

Un système de traduction automatique a besoin d'un vocaculaire pour extraire un sous langage qui soit plus facile à manipuler. 
<span style="color:red"> C'est à dire? Le vocabulaire permet d'associer à chaque mot un idéntifiant unique et un vecteur qui le represente et dont les valuers seronts appris par le réseau à travers l'entrainement. </span>


On utilise un vocabulaire basé sur les données d'apprentissage et non pas un vocabulaire quelconque car cela permet d'avoir une correspondance parfaite entre le vocabulaire de la langue source et sa traduction. Cela permet aussi d'avoir un cadre plus restreint pour l'apprentissage.

**Q5** : Quels informations nous donnent les lignes
  ````
  [...] Counters src:9978
  [...] Counters tgt:8194
  ````

Cela signifie que dans le vocabulaire du langage source, on a 9978 mots différents et dans le vocabulaire du langage target, on en a 8194.

In [ ]:
!onmt_train -config config-base.yaml

[2022-02-15 11:34:19,971 INFO] Missing transforms field for train data, set to default: [].
[2022-02-15 11:34:20,161 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-15 11:34:20,161 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-15 11:34:20,162 INFO] Parsed 2 corpora from -data.
[2022-02-15 11:34:20,163 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-15 11:34:20,163 INFO] Loading vocab from text file...
[2022-02-15 11:34:20,163 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-15 11:34:20,341 INFO] Loaded src vocab has 9978 tokens.
[2022-02-15 11:34:20,347 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-15 11:34:20,571 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-15 11:34:20,576 INFO] Building fields with vocab in counters...
[2022-02-15 11:34:20,586 INFO]  * tgt vocab size: 8198.
[2022-02-15 11:34:20,599 INFO]  * src vocab size: 

**Q6** : A quoi correspond la valeur ‘acc’ affichée par le script ? Vaut-il mieux que cette valeur soit faible ou élevée à la fin de l’apprentissage ? Pourquoi ? Même question pour ‘ppl’. Le code qui permet de calculer ces variables se trouve dans le fichier [statistics.py](https://github.com/OpenNMT/OpenNMT-py/blob/master/onmt/utils/statistics.py) de OpenNMT-py.

La valeur 'acc' correspond à la précision (accuracy). C'est le pourcentage de mots corrects sur le nombre de mots total.
Il vaut donc mieux que cette valeur soit élevée à la fin de l'apprentissage car cela signifira que le modèle a bien appris et qu'il sera assez précis pour faire la traduction dans les données de test.

La valeur 'ppl' signifie perplexity. Cette valeur se réfère à l'erreur (`exp(min(loss/nb_mots, 100))`). Il vaut donc mieux que cette valeur soit faible à la fin de l'apprentissage car cela signifira que le modèle se trompe peu dans la traduction des mots. <span style="color:red"> Intuition dérrière la définition de perplexity? </span>


**Q7** : Ouvrez le fichier de configuration _config-base.yaml_ et observez son contenu.
Décrivez en détail à quoi correspondent les paramètres suivants (n'hésitez pas à demander a Google!):
  - train_step : nombre de pas d'apprentissage <span style="color:red">C'est quoi un pas d'apprentissage? (vous n'avez que traduit le nom de la variable sans expliquer sa signification). train_step = nombre total d'étapes d'optimisation des poids du modèle, où chaque étape est constituée d'un passage en avant d'un lot (batch) de données d'apprentissage dans le modèle (forward pass) et d'un passage en arrière des gradients générés par la fonction de perte (backward pass), suivi par une mise à jours des poids du modèle. </span>
  - valid_step : effectue la validation tous les 'valid_step' fois
  - enc_layers : nombre de couches dans l'encoder
  - dec_layers : nombre de couches dans le décodeur
  - enc_rnn_size : nombre de neurones sur une couche du réseau de l'encodeur.
  - dec_rnn_size : nombre de neurones sur une couche du réseau du décodeur.
  - batch_size : taille maximum du batch pour l'entrainement. Le batch est un paquet de données qu'on envoie dans le reseau. Nombre de phrases qui sont processées à chaque step.

##  Evaluation d’un modèle de TA

In [ ]:
!onmt_translate -model models/base/model_step_625.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Traduction/pred_625.txt
!onmt_translate -model models/base/model_step_1250.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Traduction/pred_1250.txt
!onmt_translate -model models/base/model_step_1875.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Traduction/pred_1875.txt
!onmt_translate -model models/base/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Traduction/pred_2000.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-15 11:35:32,478 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-15 11:35:39,285 INFO] PRED AVG SCORE: -1.2937, PRED PPL: 3.6461
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout opti

**Q8** : Entre quelle valeur et quelle valeur est compris un score BLEU ? Vaut-il mieux que la valeur obtenue soit élevée ou faible ? Pourquoi ?

Un score BLEU est compris entre 0 et 1 (ou 0 et 100% s'il est rapporté en pourcentage).
Il vaut mieux que la valeur obtenue soit élevée (proche de 1). Plus on est proche de 1, plus les phrases sont similaires.  <span style="color:red"> Similaires à quoi? Est-ce qu'un score BLEU plus élévé veut toujours dire que la trad est méilleure ? -> non</span>

**Q9** : Pour chacun des modèles intermédiaires, calculer un score BLEU à l'aide du script `multi-bleu.perl`. Quel modèle a le meilleur score BLEU ?

In [ ]:
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Traduction/pred_625.txt
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Traduction/pred_1250.txt
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Traduction/pred_1875.txt
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Traduction/pred_2000.txt

BLEU = 6.29, 29.6/9.2/4.5/1.3 (BP=1.000, ratio=1.262, hyp_len=4509, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 13.83, 51.0/21.0/12.3/5.4 (BP=0.847, ratio=0.858, hyp_len=3065, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 17.94, 54.6/25.8/15.9/8.0 (BP=0.871, ratio=0.878, hyp_

Le meilleur score BLEU est celui du modèle model_step_2000.pt (18,48%).

## Optimisation des paramètres

**Q10 - nombre de training steps** : Augmenter le nombre de steps permet-il d’obtenir un meilleur score BLEU ? Pourquoi? Jusqu’à combien
de steps ? comment utiliser l'option `early7.2.3_stopping` ([voir ici](https://opennmt.net/OpenNMT-py/options/train.html)) pour trouver le nombre optimal de steps ?

Nous enregistrons ces modèles dans `models/nb_steps/`.

In [ ]:
!onmt_train -config config-nb-steps.yaml -early_stopping 2

[2022-02-19 09:43:14,222 INFO] Missing transforms field for train data, set to default: [].
[2022-02-19 09:43:14,223 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-19 09:43:14,223 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-19 09:43:14,224 INFO] Parsed 2 corpora from -data.
[2022-02-19 09:43:14,224 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-19 09:43:14,224 INFO] Loading vocab from text file...
[2022-02-19 09:43:14,225 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-19 09:43:14,258 INFO] Loaded src vocab has 9978 tokens.
[2022-02-19 09:43:14,265 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-19 09:43:14,281 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-19 09:43:14,285 INFO] Building fields with vocab in counters...
[2022-02-19 09:43:14,293 INFO]  * tgt vocab size: 8198.
[2022-02-19 09:43:14,305 INFO]  * src vocab size: 

L'option early_stopping permet d'arrêter l'apprentissage lorsqu'il n'y a plus d'amélioration de la précision et de l'erreur dans les prochaines validation. Par défaut l'option early_stopping concerne l'accuracy et la perplexity (ppl), mais l'option early_stopping_criteria peut nous permettre de changer ces paramètres. 

Nous avons choisi de garder ces paramètres et nous avons choisi un early_stopping de 2 (si l'accuracy est moins bonne que la meilleur accuracy deux fois de suite, l'entrainement s'arrête). Cette option nous donne aussi le meilleur modèle, ce qui nous permet donc de trouver le nombre optimal de steps (cependant, cela nécessite de bien choisir le paramètre early_stopping).

Ici, le meilleur modèle est celui à 7500 steps.

In [ ]:
!onmt_translate -model models/nb-steps/model_step_6250.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Traduction/nb-steps/pred_6250.txt
!onmt_translate -model models/nb-steps/model_step_6875.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Traduction/nb-steps/pred_6875.txt
!onmt_translate -model models/nb-steps/model_step_7500.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Traduction/nb-steps/pred_7500.txt
!onmt_translate -model models/nb-steps/model_step_8125.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Traduction/nb-steps/pred_8125.txt
!onmt_translate -model models/nb-steps/model_step_8750.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Traduction/nb-steps/pred_8750.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-19 09:51:48,621 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-19 09:51:52,328 INFO] PRED AVG SCORE: -0.5936, PRED PPL: 1.8104
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout opti

In [ ]:
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Traduction/nb-steps/pred_6250.txt
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Traduction/nb-steps/pred_6875.txt
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Traduction/nb-steps/pred_7500.txt
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Traduction/nb-steps/pred_8125.txt
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Traduction/nb-steps/pred_8750.txt

BLEU = 32.29, 64.0/39.2/27.4/18.6 (BP=0.960, ratio=0.961, hyp_len=3434, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 31.41, 64.1/39.0/26.9/17.4 (BP=0.956, ratio=0.957, hyp_len=3421, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.
BLEU = 33.67, 65.4/40.6/29.3/20.4 (BP=0.948, ratio=0.949

Nous pouvons observer que le fait d'augmenter le nombre de steps, permet d'augmenter le score BLEU dans un premier temps. Cela s'explique car on apprend plus longtemps sur nos données. Cependant, au bout d'un certain nombre de steps, le score BLEU se stabilise et oscille car il y a de l'overfitting. 

Mais augmenter le nombre de steps ne suffit pas, le score BLEU n'est toujours pas excellent.

**Q11 - nombre de couches** : Augmentez le nombre de couches de l’encodeur et du décodeur (n’allez pas au delà de 3 couches). Cela permet-il toujours d’obtenir des meilleurs résultats ? Pourquoi ?

In [ ]:
!onmt_train -config config-nb-layer.yaml

[2022-02-19 10:24:25,820 INFO] Missing transforms field for train data, set to default: [].
[2022-02-19 10:24:25,821 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-19 10:24:25,821 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-19 10:24:25,821 INFO] Parsed 2 corpora from -data.
[2022-02-19 10:24:25,822 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-19 10:24:25,822 INFO] Loading vocab from text file...
[2022-02-19 10:24:25,822 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-19 10:24:25,844 INFO] Loaded src vocab has 9978 tokens.
[2022-02-19 10:24:25,848 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-19 10:24:25,863 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-19 10:24:25,867 INFO] Building fields with vocab in counters...
[2022-02-19 10:24:25,875 INFO]  * tgt vocab size: 8198.
[2022-02-19 10:24:25,887 INFO]  * src vocab size: 

In [ ]:
!onmt_translate -model models/nb-layer/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Traduction/nb-layer/pred_2000.txt

[2022-02-19 10:25:14,049 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-19 10:25:17,945 INFO] PRED AVG SCORE: -0.9744, PRED PPL: 2.6495


In [ ]:
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Traduction/nb-layer/pred_2000.txt

BLEU = 19.15, 53.5/25.3/15.4/7.9 (BP=0.950, ratio=0.951, hyp_len=3399, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.


Augmenter le nombre de couches ne permet pas toujours d'obtenir un meilleur score BLEU car on a seulement un élément en entrée et un en sortie. Donc la fonction est linéaire et ne nécessite donc pas beaucoup de couches. 

En effet, nous avons testé avec 2 couches pour l'encodeur et pour le décodeur, 3 couches pour l'encodeur et le décodeur et 3 couches pour l'encodeur et 2 pour le décodeur. Pour ces trois exemples, le score BLEU était moins bon qu'avec 1 couche pour l'encodeur et le décodeur.

Le seul exemple ayant augmenté le score BLEU est avec 3 couches pour l'encodeur et 1 pour le décodeur. On est passé de 18,48% à 19,15%, ce qui n'est pas une augmentation très importante.

<span style="color:red">Il aurait fallu utiliser l'option early_stopping pour bien repondre à cette question (ainsi que la prochaine). En effet, si l'on utilise pas cette option, on risque de comparer des modèles qui sont sous-entrainé (c'est votre cas) ou sur-entrainés (overfitted),  sans donc pouvoir en tirer des conclusions. En effet, le nombre d'étapes d'entrainement nécessaires à l'optimization du modèle varie selon la quantité de paramétres (poids) à optimizer, ce qui est le cas lorsqu'on varie le nombre de couches ou leur taille.  </span>


**Q12 - nombre d'unités** : Faites varier le nombre d’unités pour l’encoder et le décodeur (n’allez pas au delà de 512.
Obtenez-vous de meilleurs résultats ? Pourquoi ?

In [ ]:
!onmt_train -config config-nb-unite.yaml

[2022-02-19 10:34:10,456 INFO] Missing transforms field for train data, set to default: [].
[2022-02-19 10:34:10,458 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-19 10:34:10,458 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-19 10:34:10,458 INFO] Parsed 2 corpora from -data.
[2022-02-19 10:34:10,459 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-19 10:34:10,459 INFO] Loading vocab from text file...
[2022-02-19 10:34:10,459 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-19 10:34:10,480 INFO] Loaded src vocab has 9978 tokens.
[2022-02-19 10:34:10,485 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-19 10:34:10,500 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-19 10:34:10,504 INFO] Building fields with vocab in counters...
[2022-02-19 10:34:10,512 INFO]  * tgt vocab size: 8198.
[2022-02-19 10:34:10,525 INFO]  * src vocab size: 

In [ ]:
!onmt_translate -model models/nb-unite/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Traduction/nb-unite/pred_2000.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-19 10:48:19,479 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-19 10:48:26,006 INFO] PRED AVG SCORE: -0.9050, PRED PPL: 2.4718


In [ ]:
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Traduction/nb-unite/pred_2000.txt

BLEU = 19.69, 52.3/25.3/15.6/7.9 (BP=0.978, ratio=0.978, hyp_len=3497, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.


Augmenter le nombre de neurones des couches de l'encodeur et du décodeur permet d'obtenir de meilleurs score BLEU. En effet, cela permet d'augmenter le nombre de paramètres avec lesquels jouer, à mettre à jour pour améliorer le modèle.

En effet, avec 256 unités, on avait un score BLEU de 18,48 %.
Avec 128, on obtient un score BLEU de 13,59 %.

En passant à 512, on obtient un score BLEU de 19,69%.


**Q14** : Expliquez ce qu’est le beam search. Quelle est la taille par défaut du beam ?

Le beam search (recherche en faisceau) est un algorithme de recherche heuristique qui n'explore qu'un nombre limité de fils de noeuds dans un graphe. La recherche en faisceau utilise l'algorithme de parcours en largeur. La largeur du faisceau détermine combien de fils sont explorés.

<span style="color:red"> C'est quoi les noeuds et les fils dans notre cas ? --> les tokens de chaque étape de décodage de la phrase cible, c.à.d. de la traduction </span>


La taille par défaut du beam (faisceau) est 5.

**Q15** : Changez la taille du beam en la faisant varier de 1 à 10. Comment évolue le score BLEU ?

In [ ]:
!onmt_train -config config-nb-unite.yaml

[2022-02-19 10:45:23,891 INFO] Missing transforms field for train data, set to default: [].
[2022-02-19 10:45:23,892 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-19 10:45:23,893 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-19 10:45:23,893 INFO] Parsed 2 corpora from -data.
[2022-02-19 10:45:23,894 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-19 10:45:23,894 INFO] Loading vocab from text file...
[2022-02-19 10:45:23,894 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-19 10:45:23,916 INFO] Loaded src vocab has 9978 tokens.
[2022-02-19 10:45:23,920 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-19 10:45:23,936 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-19 10:45:23,940 INFO] Building fields with vocab in counters...
[2022-02-19 10:45:23,949 INFO]  * tgt vocab size: 8198.
[2022-02-19 10:45:23,960 INFO]  * src vocab size: 

In [ ]:
!onmt_translate -beam_size 3 -model models/base/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Traduction/beam/pred_2000.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-19 10:56:50,912 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-19 10:56:53,249 INFO] PRED AVG SCORE: -1.0046, PRED PPL: 2.7308


In [ ]:
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Traduction/beam/pred_2000.txt

BLEU = 18.92, 54.7/26.0/16.2/8.0 (BP=0.914, ratio=0.917, hyp_len=3279, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.


On observe le meilleur score BLEU pour beam_size = 3.
Le score BLEU augmente entre 1 et 3 puis diminue entre 3 et 10. (sauf une valeur étrange pour beam_size = 6 où le score BLEU est de 18,24% contre 18,40% pour beam_size = 7).

<span style="color:red"> Pourquoi? </span>


**Q16** Quelle est le meilleur score BLEU que vous pouvez obtenir sur dev ? Maintenant, vous pouvez tester votre meilleur modèle en calculant son score BLEU sur l'ensemble de test. Quel résultat vous obtenez?

On prend un nombre de steps de 20000 pour être large et on ajoute l'option early_stopping à 3. On met 3 couches pour l'encodeur et 1 couche pour le décodeur, 512 unités pour l'encodeur et pour le décodeur et on met la taille du beam à 3.

In [ ]:
!onmt_train -config config-best.yaml -early_stopping 3

[2022-02-19 11:15:23,006 INFO] Missing transforms field for train data, set to default: [].
[2022-02-19 11:15:23,007 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-19 11:15:23,007 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-19 11:15:23,008 INFO] Parsed 2 corpora from -data.
[2022-02-19 11:15:23,008 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-19 11:15:23,008 INFO] Loading vocab from text file...
[2022-02-19 11:15:23,008 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-19 11:15:23,028 INFO] Loaded src vocab has 9978 tokens.
[2022-02-19 11:15:23,033 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-19 11:15:23,049 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-19 11:15:23,053 INFO] Building fields with vocab in counters...
[2022-02-19 11:15:23,061 INFO]  * tgt vocab size: 8198.
[2022-02-19 11:15:23,074 INFO]  * src vocab size: 

D'après early_stopping, le meilleur modèle est avec 8750 steps.

In [ ]:
!onmt_translate -beam_size 3 -model models/best/model_step_8750.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Traduction/best/pred_8750.txt

[2022-02-19 11:20:42,962 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-19 11:20:47,920 INFO] PRED AVG SCORE: -0.4651, PRED PPL: 1.5922


In [ ]:
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Traduction/best/pred_8750.txt

BLEU = 33.98, 64.2/39.7/29.0/20.2 (BP=0.972, ratio=0.973, hyp_len=3476, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.


On obtient un score BLEU de 33,98%.

Nous savons qu'il existe sûrement un meilleur jeu de paramètres qui permettrait d'avoir un meilleur score BLEU. Cependant cela est compliqué à réaliser à la main et cela nécessiterait des techniques de cross validation.

<span style="color:red"> La cross validation est nécessaire dans le cas où l'on a pas d'ensemble de validation. Dans notre cas nous avons bien un valid set, et donc on peut bien faire un recherche des hyperparametres plus convenant pour notre réseau. </span>





In [4]:
!onmt_translate -beam_size 3 -model models/best/model_step_8750.pt -src BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.fr.tok.txt -output Traduction/best/pred_dev_8750.txt

[2022-02-26 09:35:02,804 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-26 09:35:09,726 INFO] PRED AVG SCORE: -0.4570, PRED PPL: 1.5793


In [6]:
!perl multi-bleu.perl BTEC-en-fr/dev/IWSLT10.devset1_CSTAR03.en.tok.txt < Traduction/best/pred_dev_8750.txt

BLEU = 35.69, 66.5/42.2/30.3/21.7 (BP=0.968, ratio=0.969, hyp_len=3689, ref_len=3808)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.


On voit qu'on obtient un score BLEU de 35,69 % ce qui est meilleur que le score avec le test. Cela est logique car le vocabulaire de dev est semblable à celui du train alors que dans le test, certains mots n'auront jamais été vus.

**Q17** Observez les prédictions de votre meilleur modèle. Avez-vous l'impression que la qualité des traductions générées par ce modèle sont meilleures de celles générées par le premier modèle que vous aviez entraîné à la question Q6 ? Donnez quelque exemple. 

On observe que les prédictions sont meilleures avec notre modèle.
Voici un tableau avec quelques exemples :

| Attendu  | Modèle de base          | Notre modèle |
| :--------------- |:---------------| :-----|
| Every ten minutes.  |   All right.        |  Every ten minutes. |
| Who is speaking, please?  | Who is the way, please?             |   Who is speaking, please? |
| What kind of leather is it?  | What kind of food is it?         |    What kind of leather? |
| Do I need a reservation?  | Do I have to change?     |    Do I have to make a reservation? |


**Q13 - attention [OPTIONNEL]** : Les architectures avec mécanismes d’attention sont devenus la norme dans de nombreuses applications comme la traduction, la reconnaissance de la parole, la description d’image, etc. ([cet article de blog constitue une excellente introduction en la matière](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)). Ajouter un mécanisme d’attention en changeant le paramètre de _global_attention_ dans le fichier de configuration en le passant de _none_ à _mlp_. Observez-vous un gain de performance ? Pourquoi ? Comment-est ce que c'est mécanisme fonctionne ?

On réalise cette question optionnelle à la fin. On revient donc au modèle de base pour comparer les changements de paramètre un par un.

In [16]:
!onmt_train -config config-base-attention.yaml

[2022-02-26 10:17:09,336 INFO] Missing transforms field for train data, set to default: [].
[2022-02-26 10:17:09,340 WARNING] Corpus train's weight should be given. We default it to 1 for you.
[2022-02-26 10:17:09,340 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-26 10:17:09,343 INFO] Parsed 2 corpora from -data.
[2022-02-26 10:17:09,344 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-02-26 10:17:09,344 INFO] Loading vocab from text file...
[2022-02-26 10:17:09,344 INFO] Loading src vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.src
[2022-02-26 10:17:09,370 INFO] Loaded src vocab has 9978 tokens.
[2022-02-26 10:17:09,384 INFO] Loading tgt vocabulary from ./BTEC-en-fr/BTEC-en-fr/vocab.tgt
[2022-02-26 10:17:09,406 INFO] Loaded tgt vocab has 8194 tokens.
[2022-02-26 10:17:09,411 INFO] Building fields with vocab in counters...
[2022-02-26 10:17:09,420 INFO]  * tgt vocab size: 8198.
[2022-02-26 10:17:09,433 INFO]  * src vocab size: 

In [17]:
!onmt_translate -beam_size 3 -model models/attention/model_step_2000.pt -src BTEC-en-fr/test/IWSLT09_BTEC.testset.fr.tok.txt -output Traduction/attention/pred_2000.txt

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
[2022-02-26 10:18:50,342 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-02-26 10:18:58,933 INFO] PRED AVG SCORE: -1.2705, PRED PPL: 3.5626


In [18]:
!perl multi-bleu.perl BTEC-en-fr/test/IWSLT09_BTEC.testset.en.tok.txt < Traduction/attention/pred_2000.txt

BLEU = 9.25, 28.8/12.8/6.9/2.9 (BP=1.000, ratio=1.586, hyp_len=5668, ref_len=3574)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl can still be used for internal purposes when you have a consistent tokenizer.


On n'observe pas de gain de performance, on a même une diminution du score BLEU (de 18,48% à 9.25%). Une hypothèse serait selon nous qu'il faut mettre en place d'autres configurations pour que cette technique améliore la performance.

Ce mécanisme consiste à se focaliser sur des aspects en particulier. L'attention va aider à avoir un contexte pour chacun des mots.

<span style="color:red"> Bravo pour avoir essayé! Vous n'observé pas d'amélioration car votre modèle est sous entrainé (vous entrainé pour 2000 steps seulements). En effet, en rajoutant le mechanisme d'attention on rajoute des paramètres au modèle, ce qui implique que notre modèle aura besoin de plus de temps d'entrainement. </span>
